In [1]:
# from randtest import randtest
import numpy as np
import pandas
import pickle
from tqdm import tqdm
import random
import tensorflow.keras.backend as K

2024-02-07 13:43:07.833183: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-07 13:43:07.877853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-07 13:43:08.854116: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
print(cores)

80


In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
def forecast_loss(y_true, y_pred):
    V=134
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

In [5]:
def score(y_preds, y_true):
    y_pred = y_preds.reshape(-1, 134)
    mmse = forecast_loss(y_true, y_pred)
    s = 0
    for i in mmse:
        s += i
    return s/len(mmse)

In [6]:
def ra_test(a_0, b_0, y_true, r=1000):
    """randomization test"""
    if len(a_0) != len(b_0):
        print('Input A and B are not eaqul-sized!')
        return
    delta_s_rs = []
    c = 0
    n = len(a_0)
    
    
    delta_s_0 = score(a_0, y_true) - score(b_0, y_true)
    delta_s_0 = delta_s_0.numpy()
    
    for _ in tqdm(range(r)):
        a_r = []
        b_r = []
        # shuffle
        for i in range(n):
            # swap
            rn = random.randint(0, 1)
            # swap
            if rn == 0:
                a_r.append(b_0[i])
                b_r.append(a_0[i])
            # do not swap
            else:
                a_r.append(a_0[i])
                b_r.append(b_0[i])

        a_r = np.array(a_r)
        b_r = np.array(b_r)
        delta_s_r = score(a_r, y_true) - score(b_r, y_true)
        delta_s_r = delta_s_r.numpy()
        # print(delta_s_r)
        # print(abs(delta_s_r))
        delta_s_rs.append(delta_s_r)
        if abs(delta_s_r) >= abs(delta_s_0):
            c += 1
        
    print(c)
    p = c/r
    print(delta_s_rs)
    return p

In [7]:
baseline_path = 'randomization_test/data/baseline.pkl'
baseline_data, var_to_ind = pickle.load(open(baseline_path, 'rb'))

forecasting_preds = []

# y_true for score
y_true = []
for y in baseline_data['forecasting_test_op']:
    y_true.append(y)
y_true = np.array(y_true)

for pred in baseline_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)
x = forecasting_preds.flatten()
del baseline_data, forecasting_preds

In [8]:
m_path = 'randomization_test/data/q.pkl'
m_data, var_to_ind_m = pickle.load(open(m_path, 'rb'))
m_data

forecasting_preds = []

for pred in m_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)

y = forecasting_preds.flatten()
del m_data, forecasting_preds

In [9]:
p = ra_test(x, y, y_true = y_true, r=100)
p

2024-02-07 13:43:24.449487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31133 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0
2024-02-07 13:43:24.450099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31133 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2024-02-07 13:43:24.450583: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31132 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0
2024-02-07 13:43:24.451071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 31093 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id

0
[0.013315891838302285, -0.004048648783548714, -0.0041472115667238185, 0.006990736288887689, -0.0036727283288646717, 0.0117097382126774, -0.0007765551803853299, -0.008197311961638754, -0.00826926796103411, 0.006067661204118835, -0.0018702656192424172, 0.007587564720314965, 0.005995579499496806, 0.0011161884083152884, -0.03803264957163588, -0.005656828002629233, 0.008793879049802555, -0.0034130990133629524, -0.0082469970572987, 0.017556619364621717, -0.005100056095116656, -0.02206312692229684, -0.01949410900484949, 0.014003039627534797, 0.0032474755073392814, 8.076072646989729e-05, 0.010371685597848845, 0.012984133610512316, -0.0015830059500201443, -0.010754674298038225, -0.002395725174032748, -0.008906059304636393, -0.0012585905031157196, 0.01923378625143357, 0.003648058917468333, 0.010532528276047337, -0.005442355812187216, -0.0029809712437680602, 0.007131876424285366, -0.01939559042562511, 0.010470092902606254, 0.00415731950949727, -0.009582210676879477, 0.0172526475027297, -0.00063

0.0